<a href="https://colab.research.google.com/github/MODA-NYC/nyc-geography-crosswalks/blob/main/NYC_Geography_Crosswalks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proof of Concept: Community District Crosswalk Generation

This proof of concept generates sample crosswalks with Community Districts (CD) as the base using spatial filters to compute and refine the overlaps between CDs and various other geography layers. The calculations are performed using the `all_bounds.geojson` file from [BetaNYC/nyc-boundaries](https://github.com/BetaNYC/nyc-boundaries).

## Example Community District Crosswalk (wide)

**Description**  
This cell generates a **wide** crosswalk table where each row represents a **single Community District** and each column corresponds to a **unique geography ID**. We apply a **negative buffer** (in feet) to each Community District boundary to exclude abutting (touching) features, and we also require a **minimum intersection area** (in square feet) to filter out trivial overlaps. The result is a table with semicolon‐delimited lists of `nameCol` values in each cell, indicating which features (from each ID) truly overlap each Community District after these spatial filters.

**Output**  
A **DataFrame** with one row per Community District and columns for each geography ID. Cells show the **nameCol** values of the overlapping features (semicolon‐delimited). Any cell remains blank if no features for that ID meet the buffer and intersection area requirements.


In [ ]:
# Install required libraries if needed
!pip install geopandas requests --quiet

import geopandas as gpd
import pandas as pd
import requests
from io import BytesIO

# ---------------------------------------------------------
# CONFIGURATION
# ---------------------------------------------------------
# Negative buffer value (in feet); this shrinks the CD geometry.
BUFFER_FEET = -200
# Minimum required intersection area (in square feet) after buffering.
MIN_INTERSECTION_AREA = 400

# Desired list of IDs as columns
desired_ids = [
    'pp','fb','sd','bid','ibz','cd','dsny','hc','cc_upcoming',
    'cc','nycongress','sa','ss','nta','zipcode','hd'
]

# ---------------------------------------------------------
# STEP 1: DOWNLOAD AND READ THE GEOJSON
# ---------------------------------------------------------
geojson_url = "https://raw.githubusercontent.com/BetaNYC/nyc-boundaries/main/script/all_bounds.geojson"
response = requests.get(geojson_url)
if response.status_code == 200:
    print("GeoJSON file successfully downloaded.")
    gdf = gpd.read_file(BytesIO(response.content))
else:
    raise Exception("Failed to download the GeoJSON file. Check the URL or internet connection.")

print("Columns in the GeoDataFrame:", gdf.columns.tolist())
print("Sample rows:\n", gdf.head(), "\n")

# ---------------------------------------------------------
# STEP 2: REPROJECT TO A LOCAL PROJECTION (EPSG:2263)
#         FOR CONSISTENT DISTANCE-BASED OPERATIONS
# ---------------------------------------------------------
if gdf.crs is None:
    # Assume WGS84 if unknown
    gdf.set_crs(epsg=4326, inplace=True)

# Reproject to EPSG:2263 (units = feet)
gdf = gdf.to_crs(epsg=2263)

# ---------------------------------------------------------
# STEP 3: SELECT COMMUNITY DISTRICTS (Rows)
#         We assume features with id=='cd' represent Community Districts.
# ---------------------------------------------------------
cd_gdf = gdf[gdf['id'] == 'cd'].copy()
print(f"Found {len(cd_gdf)} features where id='cd'.")

# ---------------------------------------------------------
# STEP 4: DETERMINE WHICH DESIRED IDs ARE PRESENT (Columns)
# ---------------------------------------------------------
all_ids_in_data = set(gdf['id'].unique())
found_ids = [i for i in desired_ids if i in all_ids_in_data]

print("Unique 'id' values in the dataset:", gdf['id'].unique())
print("We'll create columns for these IDs:", found_ids, "\n")

# ---------------------------------------------------------
# STEP 5: BUILD A SPATIAL INDEX ON THE ENTIRE DATASET
# ---------------------------------------------------------
all_sindex = gdf.sindex

# ---------------------------------------------------------
# STEP 6: BUILD THE CROSSWALK USING THE COMBINED APPROACH
#   - For each Community District, we first create a negative buffer.
#   - Then, we get candidate features (via bounding box) and filter:
#       (a) They must intersect the negative-buffered geometry.
#       (b) Their intersection area (with the negative-buffered geometry)
#           must be greater than MIN_INTERSECTION_AREA.
# ---------------------------------------------------------
crosswalk_records = []

for idx, cd_row in cd_gdf.iterrows():
    cd_name = cd_row['nameCol']  # Community District name from the cd feature
    cd_geom = cd_row.geometry
    # Apply a negative buffer to shrink the geometry and exclude mere abutments.
    cd_geom_buffered = cd_geom.buffer(BUFFER_FEET)

    # Get candidate features using the bounding box of the buffered geometry.
    candidate_idx = list(all_sindex.intersection(cd_geom_buffered.bounds))
    candidate_features = gdf.iloc[candidate_idx]

    # First filter: keep those that intersect the negative-buffered geometry.
    mask = candidate_features.intersects(cd_geom_buffered)
    candidates = candidate_features[mask].copy()

    # Second filter: calculate the intersection area with the negative-buffered geometry.
    # (This will be small or zero for features that only abut.)
    if not candidates.empty:
        candidates["intersection_area"] = candidates.geometry.intersection(cd_geom_buffered).area
        # Keep only candidates with a sufficient intersection area.
        final_candidates = candidates[candidates["intersection_area"] > MIN_INTERSECTION_AREA]
    else:
        final_candidates = candidates  # empty GeoDataFrame

    # Prepare the record for this Community District.
    record = {'Community District': cd_name}

    # For each desired id column, join the 'nameCol' values of the intersecting features.
    for the_id in found_ids:
        subset = final_candidates[final_candidates['id'] == the_id]
        if not subset.empty:
            namecols = subset['nameCol'].unique()
            record[the_id] = ";".join(map(str, namecols))
        else:
            record[the_id] = ""

    crosswalk_records.append(record)

# Create the wide-format DataFrame.
crosswalk_df = pd.DataFrame(crosswalk_records)
col_order = ['Community District'] + found_ids
crosswalk_df = crosswalk_df[col_order]

print("Crosswalk DataFrame (first 5 rows):")
print(crosswalk_df.head(5))

# Optionally, save to CSV:
# crosswalk_df.to_csv('crosswalk_combined_filter.csv', index=False)


GeoJSON file successfully downloaded.
Columns in the GeoDataFrame: ['id', 'nameCol', 'nameAlt', '_errors', 'layer', 'path', 'geometry']
Sample rows:
     id            nameCol nameAlt                 _errors           layer  \
0   pp                 63    None  Ring self-intersection  Invalid output   
1   fb                 53    None  Ring self-intersection  Invalid output   
2   sd                 17    None  Ring self-intersection  Invalid output   
3  bid  Myrtle Avenue DMA    None  Ring self-intersection  Invalid output   
4  ibz     North Brooklyn    None  Ring self-intersection  Invalid output   

                                                path  \
0  Polygon?crs=EPSG:4326&field=id:string(0,0)&fie...   
1  Polygon?crs=EPSG:4326&field=id:string(0,0)&fie...   
2  Polygon?crs=EPSG:4326&field=id:string(0,0)&fie...   
3  Polygon?crs=EPSG:4326&field=id:string(0,0)&fie...   
4  Polygon?crs=EPSG:4326&field=id:string(0,0)&fie...   

                                            geomet

In [ ]:
crosswalk_df.head()

,Community District,pp,fb,sd,bid,ibz,cd,dsny,hc,cc_upcoming,cc,nycongress,sa,ss,nta,zipcode,hd
0,404,110,46,28;24,82nd Street BID,,404,QW04,46;42,30;25;21,25;21,6;14,39;35;30;34,13;12,Elmhurst;Corona;North Corona;Elmhurst-Maspeth,11373;11368;11377,
1,304,83,57;44;37;28,14;19;23;32,,Ridgewood;North Brooklyn,304,BKN04,34,37;34,37;34,7,54;53,25;18;12,park-cemetery-etc-Brooklyn;Bushwick North;Bush...,11206;11207;11237;11221,
2,303,79;81,38;57;31;44;37,13;14;16,Bed-Stuy Gateway?BID,,303,BKN03,36;32;34,36;33;41,35;36;33;41,8;7,57;55;56,20;25,Crown Heights North;Bedford;Clinton Hill;Ocean...,11238;11216;11206;11205;11213;11233;11221,Bedford Historic District;Alice and Agate Cour...
3,308,78;77,38;57,17;13;16,,,308,BKN08,36;38;32,35;36;39;41,35;36;39;41,9;10;8;7,43;44;57;52;55;56,20;25,Crown Heights North;Prospect Heights;Park Slop...,11217;11238;11216;11213;11233,Crown Heights North II Historic District;Crown...
4,112,33;34,13,6,Washington Heights BID,,112,MN12,17,7;10,7;10,13,71;72,31,park-cemetery-etc-Manhattan;Washington Heights...,10031;10032;10033;10040;10034,Audubon Terrace Historic District;Audubon Park...


## Example Community District Crosswalk (long-form)

**Description**  
In this cell, we produce a **long‐form** table where each row corresponds to a **unique combination** of a Community District and another feature’s `nameCol` (e.g., a specific school district or BID name). We again apply the **negative buffer** and **minimum intersection area** thresholds to identify valid overlaps. Then, for each valid overlap, we compute the **area** of intersection with the **original** (non‐buffered) Community District and express it as a **percentage** of the Community District’s total area. This method ensures we see the precise share of each CD covered by each overlapping geography.

**Output**  
A **DataFrame** with columns for the **Community District** name, the **other geography ID**, the **specific `nameCol`** within that ID, the **CD area**, the **intersection area**, and the **percentage overlap**. Each row is a single CD + other geography combination, making it easier to see exactly how much of a CD is covered by a particular boundary.

In [ ]:
# Install required libraries if needed
!pip install geopandas shapely requests --quiet

import geopandas as gpd
import pandas as pd
import requests
from io import BytesIO

# For Shapely 2.0 union_all() method
try:
    from shapely.ops import unary_union  # fallback if union_all not available
    HAS_UNION_ALL = hasattr(gpd.GeoSeries([]).geometry, 'union_all')
except ImportError:
    HAS_UNION_ALL = False

# ---------------------------------------------------------
# CONFIGURATION
# ---------------------------------------------------------
BUFFER_FEET = -200          # Negative buffer to shrink CDs
MIN_INTERSECTION_AREA = 40  # sq ft threshold

# Desired list of IDs to consider as "other geographies"
desired_ids = [
    'pp','fb','sd','bid','ibz','cd','dsny','hc','cc_upcoming',
    'cc','nycongress','sa','ss','nta','zipcode','hd'
]

# ---------------------------------------------------------
# STEP 1: DOWNLOAD AND READ THE GEOJSON
# ---------------------------------------------------------
geojson_url = "https://raw.githubusercontent.com/BetaNYC/nyc-boundaries/main/script/all_bounds.geojson"
response = requests.get(geojson_url)
if response.status_code == 200:
    print("GeoJSON file successfully downloaded.")
    gdf = gpd.read_file(BytesIO(response.content))
else:
    raise Exception("Failed to download the GeoJSON file. Check the URL or internet connection.")

# ---------------------------------------------------------
# STEP 2: REPROJECT TO A LOCAL PROJECTION (EPSG:2263)
# ---------------------------------------------------------
if gdf.crs is None:
    gdf.set_crs(epsg=4326, inplace=True)
gdf = gdf.to_crs(epsg=2263)

# ---------------------------------------------------------
# STEP 3: SELECT COMMUNITY DISTRICTS (id='cd')
# ---------------------------------------------------------
cd_gdf = gdf[gdf['id'] == 'cd'].copy()
print(f"Found {len(cd_gdf)} Community District features (id='cd').")

# ---------------------------------------------------------
# STEP 4: DETERMINE WHICH OTHER IDs ARE PRESENT
# ---------------------------------------------------------
all_ids_in_data = set(gdf['id'].unique())
found_ids = [i for i in desired_ids if i in all_ids_in_data]
print("IDs in dataset:", gdf['id'].unique())
print("We'll create rows for these IDs:", found_ids)

# ---------------------------------------------------------
# STEP 5: SPATIAL INDEX FOR EFFICIENCY
# ---------------------------------------------------------
all_sindex = gdf.sindex

# ---------------------------------------------------------
# STEP 6: BUILD THE LONG-FORM OVERLAP TABLE
#   - One row per (CD + other_id + unique nameCol)
#   - Negative buffer to exclude abutting geographies
#   - Minimum intersection area threshold to exclude minor overlaps
#   - Intersection area & percentage overlap based on original CD geometry
# ---------------------------------------------------------
rows = []

for idx, cd_row in cd_gdf.iterrows():
    cd_name = cd_row['nameCol']         # e.g., '404'
    cd_geom = cd_row.geometry           # original CD geometry
    cd_area = cd_geom.area

    # Create a negative-buffered geometry for filtering
    cd_geom_buffered = cd_geom.buffer(BUFFER_FEET)

    # Candidate features must intersect this buffered geometry
    candidate_idx = list(all_sindex.intersection(cd_geom_buffered.bounds))
    candidate_features = gdf.iloc[candidate_idx]

    # For each "other" ID
    for other_id in found_ids:
        if other_id == 'cd':
            # Skip self-overlaps
            continue

        # Subset to features of this ID that intersect the buffered geometry
        subset = candidate_features[
            (candidate_features['id'] == other_id) &
            (candidate_features.intersects(cd_geom_buffered))
        ].copy()

        if not subset.empty:
            # Compute intersection area with the buffered CD
            subset['intersect_area'] = subset.geometry.intersection(cd_geom_buffered).area
            # Exclude minor overlaps
            subset = subset[subset['intersect_area'] > MIN_INTERSECTION_AREA]
        else:
            subset = gpd.GeoDataFrame(columns=gdf.columns)

        # Now we have filtered features for this (CD + other_id).
        # For each unique nameCol in this subset, create one row.
        for name_val in subset['nameCol'].unique():
            feats_same_name = subset[subset['nameCol'] == name_val]

            # Union all geometries for this nameCol
            if not feats_same_name.empty:
                # If Shapely 2.0 is installed, use union_all(); otherwise fallback to unary_union
                if HAS_UNION_ALL:
                    union_geom = feats_same_name.geometry.union_all()
                else:
                    union_geom = feats_same_name.geometry.unary_union

                # Intersection with the ORIGINAL CD geometry
                inter_geom = cd_geom.intersection(union_geom)
                inter_area = inter_geom.area if not inter_geom.is_empty else 0
                perc_overlap = (inter_area / cd_area) * 100 if cd_area > 0 else 0
            else:
                inter_area = 0
                perc_overlap = 0

            row = {
                "Community District": cd_name,
                "Other Geography ID": other_id,
                "Other Geography NameCol": name_val,
                "CD Area (sq ft)": cd_area,
                "Intersection Area (sq ft)": inter_area,
                "Percentage Overlap": perc_overlap
            }
            rows.append(row)

# Create a DataFrame of the long-form overlaps
overlap_df = pd.DataFrame(rows)

print("\nOverlap table (first 25 rows):")
print(overlap_df.head(25))

# OPTIONAL: save to CSV
# overlap_df.to_csv("cd_other_overlap_longform.csv", index=False)


GeoJSON file successfully downloaded.
Found 71 Community District features (id='cd').
IDs in dataset: ['pp' 'fb' 'sd' 'bid' 'ibz' 'cd' 'dsny' 'hc' 'cc_upcoming' 'cc'
 'nycongress' 'sa' 'ss' 'nta' 'zipcode' 'hd']
We'll create rows for these IDs: ['pp', 'fb', 'sd', 'bid', 'ibz', 'cd', 'dsny', 'hc', 'cc_upcoming', 'cc', 'nycongress', 'sa', 'ss', 'nta', 'zipcode', 'hd']

Overlap table (first 25 rows):
   Community District Other Geography ID Other Geography NameCol  \
0                 404                 pp                     110   
1                 404                 fb                      46   
2                 404                 sd                      28   
3                 404                 sd                      24   
4                 404                bid         82nd Street BID   
5                 404               dsny                    QW04   
6                 404                 hc                      46   
7                 404                 hc               

In [ ]:
overlap_df.head(25)

,Community District,Other Geography ID,Other Geography NameCol,CD Area (sq ft),Intersection Area (sq ft),Percentage Overlap
0,404,pp,110,6.573964e+07,6.573964e+07,100.000000
1,404,fb,46,6.573964e+07,6.566597e+07,99.887942
2,404,sd,28,6.573964e+07,1.102672e+06,1.677331
3,404,sd,24,6.573964e+07,6.453217e+07,98.163260
4,404,bid,82nd Street BID,6.573964e+07,1.389619e+05,0.211382
5,404,dsny,QW04,6.573964e+07,6.569810e+07,99.936804
6,404,hc,46,6.573964e+07,6.958567e+06,10.585039
7,404,hc,42,6.573964e+07,5.864845e+07,89.213228
8,404,cc_upcoming,30,6.573964e+07,1.248337e+07,18.989104
9,404,cc_upcoming,25,6.573964e+07,2.662081e+07,40.494306
